<a href="https://colab.research.google.com/github/aks2093/StackExchange-Question-Tags-Prediction/blob/master/StackExachange_tags_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Post Proressing of the output tags if it contains "rare" tag

**read the data**

In [ ]:
import os
import pandas as pd

DATA_DIR = "/content/drive/My Drive/stackExchange_tag_predictions/data"
if os.path.isfile(os.path.join(DATA_DIR, "combined_data.csv")):
    combined_data = pd.read_csv(os.path.join(DATA_DIR, "combined_data.csv"))
    print("combined Data read")

combined Data read


**Get total tags and unique tags**

In [ ]:
Y = combined_data[["tags"]]

Y["tags"] = Y["tags"].apply(lambda x: x.lower().split(" "))
print(Y.head(5))
unique_tags = []
total_tags = []
for _, row in Y.iterrows():
    total_tags.extend(list(row["tags"]))
    if len(unique_tags) == 0:
        unique_tags = set(row["tags"])
    else:
        unique_tags = unique_tags.union(set(row["tags"]))

number_of_unique_tags = len(unique_tags)
print("Total number of unique tags: {}".format(number_of_unique_tags))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


                                   tags
0      [aes, pseudo-random-permutation]
1           [junction-box, low-voltage]
2  [cryptanalysis, substitution-cipher]
3            [motor, quadcopter, power]
4                  [electrical, boiler]
Total number of unique tags: 4268


# Build a probabilistic matrix


Building a Event Matrix whose rows will represent frequent_tags and columns will represent rare_tags. 

value to be filled out in this matrix would be
event_matrix[frequent_tag][rare_tag]= Number of events when frequent_tag and rare_tag occured together

Idea is to compute the probability vector for each rare_tag so that we can associate them with frequent_tag.

**Proposed Algorithm:**


> for all rare_tags:

1.   Compute P(rare_tag|freq_tag1) X P(rare_tag|freq_tag2) X........X P(rare_tag|freq_tag_n)
2.   add them to probability vector




**The Top K values in the probability vector will give the K rare_tags that can be associated with frequent_tag that was predicted by model.**


In [ ]:
import numpy as np

counter_dict = {}
for tag in total_tags:
    if tag not in counter_dict:
        counter_dict[tag] = 1
    else:
        counter_dict[tag] += 1

rare_tags = []
frequent_tags = []

for key, val in counter_dict.items():
    if val<=500:
        rare_tags.append(key)
    else:
        frequent_tags.append(key)

row_wise_tags = []
for _, row in Y.iterrows():
    row_wise_tags.append(list(row["tags"]))


In [ ]:
print("Number of rare tags: {}".format(len(rare_tags)))
print("Number of frequent tags: {}".format(len(frequent_tags)))
print("row_wise_tags length: {}".format(len(row_wise_tags)))
#Build Event Matrix
print("Computing Event Matrix.....")
event_matrix = [[0]*len(rare_tags) for i in range(len(frequent_tags))]
# numpy.zeros(shape=(len(frequent_tags),len(rare_tags)))

print("num of rows in event matrix: {}".format(len(event_matrix)))
print("num of cols in event matrix: {}".format(len(event_matrix[0])))
print("Initialization of Event Matrix Done.....")

for rare_tag_idx in range(len(rare_tags)):
    for frequent_tag_idx in range(len(frequent_tags)):
        count = 0
        for row_tags in row_wise_tags:
            if (rare_tags[rare_tag_idx] in row_tags) and (frequent_tags[frequent_tag_idx] in row_tags):
                count += 1
        event_matrix[frequent_tag_idx][rare_tag_idx] = count

event_matrix = np.array(event_matrix)
np.save(os.path.join(DATA_DIR, "event_matrix"), event_matrix)

print("Event Matrix computed")

Number of rare tags: 4203
Number of frequent tags: 65
row_wise_tags length: 87000
Computing Event Matrix.....
num of rows in event matrix: 65
num of cols in event matrix: 4203
Initialization of Event Matrix Done.....
Event Matrix computed
['communication', 'laundry', 'pregnancy', 'sex', 'primer']


**Make Prediction based on Probability model for rare_tags**

In [ ]:
import numpy as np
event_matrix = np.load(os.path.join(DATA_DIR, "event_matrix.npy"))

#Get the rare_tags based on probability
def get_rare_tags(tags, top_k=5):
    if "rare" in tags and len(tags)>=2:
        tags.remove("rare")
        probability_vector = []
        for rare_tag in rare_tags:
            p = 1.0
            for tag in tags:
                rare_tag_idx = rare_tags.index(rare_tag)
                tag_idx = frequent_tags.index(tag)
                if event_matrix[tag_idx][rare_tag_idx] != 0:
                    p = p*(event_matrix[tag_idx][rare_tag_idx]/counter_dict[tag])
            if p<1.0:
                probability_vector.append((rare_tag, p))

        probability_vector.sort(key=lambda x : x[1])
        predicted_rare_tags = []
        for i in range(top_k):
            predicted_rare_tags.append(probability_vector[i][0])

        return predicted_rare_tags
    else:
        raise Exception("No rare tag found")
    
print("Predicted top 5 rare_tags are: ")
print(get_rare_tags(["evolution", "genetics", "molecular-biology", "rare"]))



Predicted top 5 rare_tags are: 
['extremophiles', 'protein-folding', 'network', 'invertebrates', 'radiation']


**Define custom functions those were used while training the model**

In [ ]:
from keras import backend as K
import tensorflow

def wbce( y_true, y_pred, weight1=0.9, weight0=0.1 ) :
    y_true = K.clip(y_true, K.epsilon(), 1-K.epsilon())
    y_pred = K.clip(y_pred, K.epsilon(), 1-K.epsilon())
    logloss = -(y_true * K.log(y_pred) * weight1 + (1 - y_true) * K.log(1 - y_pred) * weight0 )
    return K.mean( logloss, axis=-1)

def micro_f1_score(y_true, y_pred, threshold=None):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    if threshold is None:
        threshold = tensorflow.reduce_max(y_pred, axis=-1, keepdims=True)
        y_pred = tensorflow.logical_and(y_pred >= threshold, tensorflow.abs(y_pred) > 1e-12)
    else:
        y_pred = y_pred > threshold
    
    y_true = tensorflow.cast(y_true, tensorflow.float32)
    y_pred = tensorflow.cast(y_pred, tensorflow.float32)

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def jaccard_distance(y_true, y_pred, threshold=None, smooth=100):
    if threshold is None:
        threshold = tensorflow.reduce_max(y_pred, axis=-1, keepdims=True)
        y_pred = tensorflow.logical_and(y_pred >= threshold, tensorflow.abs(y_pred) > 1e-12)
    else:
        y_pred = y_pred > threshold
    
    y_true = tensorflow.cast(y_true, tensorflow.int32)
    y_pred = tensorflow.cast(y_pred, tensorflow.int32)

    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac)

def hamming_loss(y_true, y_pred, threshold=None):
    if threshold is None:
        threshold = tensorflow.reduce_max(y_pred, axis=-1, keepdims=True)
        y_pred = tensorflow.logical_and(y_pred >= threshold, tensorflow.abs(y_pred) > 1e-12)
    else:
        y_pred = y_pred > threshold

    y_true = tensorflow.cast(y_true, tensorflow.int32)
    y_pred = tensorflow.cast(y_pred, tensorflow.int32)

    nonzero = tensorflow.cast(tensorflow.math.count_nonzero(y_true - y_pred, axis=-1), tensorflow.float32)
    return nonzero / y_true.get_shape()[-1]

In [ ]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MultiLabelBinarizer

X = combined_data[["title_content"]]
Y = combined_data[["tags"]]
Y["tags"] = Y["tags"].apply(lambda x: x.lower().split(" "))

counter_dict = {}
for tag in total_tags:
    if tag not in counter_dict:
        counter_dict[tag] = 1
    else:
        counter_dict[tag] += 1

rare_tags = []
for key, val in counter_dict.items():
    if val<=500:
        rare_tags.append(key)

from copy import deepcopy
def remove_rare_tags(tags):
    temp_tags = deepcopy(tags)
    is_rare = False
    for tag in temp_tags:
        if tag in rare_tags:
            is_rare = True
            tags.remove(tag)
    if is_rare:
        tags.append("rare")
    return tags

print("Number of rare tags seen: {}".format(len(rare_tags)))

Y["tags"] = Y["tags"].apply(remove_rare_tags)
mlb = MultiLabelBinarizer()
Y_bin = mlb.fit_transform(Y["tags"])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Number of rare tags seen: 4203


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(X["title_content"])
max_length = max([len(s.split()) for s in X["title_content"]])
vocab_size = len(tokenizer_obj.word_index)+1
print("vocab_size: {}".format(vocab_size))
title_content_tokens = tokenizer_obj.texts_to_sequences(X["title_content"])
title_content_token_len = [len(i) for i in title_content_tokens]
title_content_tokens_pad = pad_sequences(title_content_tokens, maxlen=max_length, padding='pre')

title_content_tokens_pad_sample = title_content_tokens_pad[45:55]
model = keras.models.load_model(os.path.join(DATA_DIR,"stackExchange_tags_model_Adam.h5"),custom_objects={'wbce':wbce,"hamming_loss":hamming_loss,"micro_f1_score":micro_f1_score,"jaccard_distance":jaccard_distance })
predictions = model.predict([title_content_tokens_pad_sample])

vocab_size: 167055


**get labels based on a given optimal threshold**

In [ ]:
def get_labels_tf(y_pred, threshold=None):
    y_pred = tensorflow.convert_to_tensor(y_pred)
    if threshold is None:
        threshold = tensorflow.reduce_max(y_pred, axis=-1, keepdims=True)
        y_pred = tensorflow.logical_and(y_pred >= threshold, tensorflow.abs(y_pred) > 1e-12)
    else:
        y_pred = y_pred > threshold
    return y_pred

def get_labels_np(y_pred, threshold=0.5):
    if threshold is None:
        threshold = np.amax(y_pred, axis=-1, keepdims=True)
        y_pred = np.logical_and(y_pred >= threshold, np.absolute(y_pred) > 1e-12)
    else:
        y_pred = y_pred > threshold
    return y_pred

# Results



The current trained model is giving following results:

**Train:**
micro_f1_score: 0.6793 - hamming_loss: 0.0129 - jaccard_distance: 0.0083 

**Test:**
val_micro_f1_score: 0.6787 - val_hamming_loss: 0.0129 - val_jaccard_distance: 0.0083

# Future Work







As we have seen that the target tags were quite sparse(shape=87000x4268) because of rare tags(frequency<=500) so Classification algorithm was not able to perform well on these many tags(labels). So we had to make a separate new category for all rare tags(count=4203) so that dimentions of the target gets reduced and resultant target matrix would be less sparse so that out algo can I dentify each label appropriately.

Although we can further improve the results with following few approaches:
1. Making a proper bucketing of rare tags based on the correlation exist among them annd then train the Deep learning model on that.
2. Take the data encodings from sencond last layer from the trained model and build a separate model that will predict those rare tags.
3. We can also predict the tags based on the correlation among them
4. skip-gram model can help to predict the rare tags based on similar context shared.

We can also use the some advanced pretrained models like **BERT** or **T5** and predict the tags based on the encodings coming from these pretrained models. Due to time constraint I cound not use these models. I would like to share another notebook based on results of these advanced models

One more nobel approach I found that reddit follows to classify its post to subreddits. 